In [1]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config

env: JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
env: FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config


In [2]:
%load_ext streaming_jupyter_integrations.magics

Could not find scli config file at '/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/.streaming_config.yml'. Will not load any secrets.
Set env variable JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'


In [3]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081


remote environment has been created.


In [4]:
%%flink_execute_sql
CREATE TABLE encounter (
  `id` STRING,
  `status` STRING,
  `type` ROW (
    `coding` ROW (
      `code` STRING,
      `display` STRING
    ) ARRAY  ) ARRAY,
  `subject` ROW (
    `reference` STRING,
    `display` STRING
  ),
  `participant` ROW (
    `individual` ROW (
      `reference` STRING,
      `display` STRING
    )
  ) ARRAY,
  `period` ROW (
    `start` STRING,
    `end` STRING
  ),
  `serviceProvider` ROW (
    `reference` STRING
  ) 
) WITH (
  'connector' = 'kafka',
  'topic' = 'Encounter',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [5]:
%%flink_execute_sql
CREATE TABLE visit_psql (
  `identifier` STRING,
  `id_organization` STRING,
  `id_practitioner` STRING,
  `id_patient` STRING,
  `status` STRING,
  `startTime` STRING,
  `endTime` STRING,
  `type` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://ep-dark-sea-a20skpa3-pooler.eu-central-1.aws.neon.tech:5432/verceldb',
  'table-name' = 'public.visit',
  'username' = 'default',
  'password' = 'tUyXvsaMq07d'
);

Job starting...
Job started
Execution successful


In [6]:
%%flink_execute_sql
INSERT INTO visit_psql
SELECT 
id as identifier, 
 REPLACE(
    e.serviceProvider.reference, 
    'Organization?identifier=https://github.com/synthetichealth/synthea|', 
    ''
  ) AS id_organization,
  REPLACE(
    e.participant[1].individual.reference, 
    'Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|', 
    ''
  ) AS id_practitioner,
  REPLACE(
      e.`subject`.`reference`, 
    'urn:uuid:', 
    ''
  )
  AS id_patient,
status,
 e.`period`.`start` as startTime,
  e.`period`.`end` as endTime,
   e.type[1].coding[1].display as `type`
   FROM encounter e;

This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Job started
